##  Missing: A section introducing Wordl and the concept of a WordlBot.

In this notebook we move a significant part of the way toward building a Wordl-bot, a program that can play Wordl significantly better than you.  The  first thing we will focus on is a way of scoring candidate guesses.  Some guesses are  much better than others because on average they knock out more contenders than other candidates.  You need a scoring metric that measures this property: `metric(candidate, all_candidates) = score`.

In the rest of this note we give a quick sketch of how a scoring metric works. The intuition
we'll try to flesh out is that the best scoring metric knocks out more possibilities than others **on average**.
But what does that mean?

We saw in the last exercise that the guess "audio" left us with 85 candidates.  And we knew that because
we executed the function `compatible_words`.  But maybe there was a guess that would
have left us with only 50 candidates.  That would have been better. 
That's an insight, but not an immediately helpful one.
How did we arrive at the information that one guess left us with 85 candidates?
We computed that by knowing the coloring and then finding the
words compatible with that coloring, but computing
a coloring requires knowing the target.
How do we score guesses when we don't know the target?

Here's what we do. We hold the guess constant and we compute
the coloring that guess would produce if each of the possible candidate words
were the target.  That gives a set of colorings, and each coloring is associated with a set
of candidate words (possibly a set of size 1).  This set of sets is called a **partition** and
the key insight is that it should be partitions we score. Each of the sets in the partition  is
one possible outcome for the set of words left after our guess.  So the
simplest idea for scoring the guess is just to compute the average size
of those partition sets.  The smaller the average size, the more words 
eliminated on average, and the better the guess.
That's not bad.  You could build a pretty good WordlBot with that idea.

But it turns out there is a better idea.  Our problem can be phrased:
How do we measure the amount of information gained in going
from the original set of candidates to a smaller set? 
If we can answer that question, then we just compute the
average of the information gain for each set in the partition
and we have a useful score for our guess.

This is exactly the sort of question answered by a branch of computer science called **Information Theory**.
The information theoretic approach often settles on the same answer
as average partition size, but it's somewhat more principled
and a lot more sophisticated because it is grounded in a 
mathematical theory for measuring the information of an event,
which is in turn grounded in a very satisfying way in probability
theory.  What follows is a **very brief** sketch of how 
something called (**information gain** (closely related to entropy)
can be used to determine which guess achieves the average gain in
information. 

#### An information-theoretic measure

We have a set of candidate words.  Choose one of them as a guess.
Consider each candidate word in turn and determine what
coloring would result if that word were the target.
For example, if the guess is "mouth" and the target 
word is "snort", the resulting coloring would be "kykyk" because
target letters "o" and "t" are contained in the guess but not in
the correct positions. The candidates "toxic", "south", and "topic" would all
get the same coloring as the guess "mouth".
So "toxic", "south",  and "topic" all
go in the same coloring set as the guess "mouth".
The colorings **partition** the set of candidates: every candidate gets a coloring and no candidate
gets more than one.  Another way of saying it is that a guess partitions
the set of candidates into a set of coloring classes.

So how does this help score a guess? Well, it does help.
Now consider comparing the partitions that result from different guesses.
Better generally means more coloring classes with
fewer members.  For example if the current set
of candidate has 24 words, the best possible partitioning would be 24 classes with
1 member each.  Because then no matter which candidate turns out to be right,
we (errorless logical agents that we are)  would take take one look at the resulting
coloring and we would be sure to guess the word on the next guess.

So how do we quantify this?  Say there are 24 candidates. What's better,
guess A which defines a partition with  2 groups of size 3, 4 groups of size 2, and 10 of size 1,
or guess B which defines a partition with 1 group of size 3, 8 groups of size 2, and 5 of size 1?

$$
\begin{array}{l|c|c}
\text{Name}& \text{Number of groups} & \text{Size}\\
\hline
\text{A} &  2 & 3\\
         &  4 & 2\\
         & 10 & 1 \\
         \hline
\text{B} & 1 & 3\\
         & 8 & 2\\
         & 5 & 1\\
         \hline
\end{array}
$$

Here's  the information theoretic way of looking at. it.
When we make a guess, it earns a coloring, and now we have a new (and smaller!) set of candidate words.
That's information gained.
We score a  guess  by computing the amount of information
gained for each of the colorings it might result in
and then taking the average of all the possible information gains.

Suppose, for example,  the game thus far has led us to a set of
candidate words which determines partitioning A above.
And suppose our next guess narrows things down to one of the two groups with 3 members.  Then a random
guess has a 1/3 chance of being the target.  Suppose,
on the other hand, we land in one of the 10 groups with 1 member.
Then a random guess has a 100% chance of being the target.
(we say the "random" guess has a probability of 1.0 of being right).
Clearly we have gained more information by landing in the seond
group than in the first.  We'd like to quantify the intuition that smaller
candidate sets are better, and and we're to do that by using the fact
that in smaller groups the probability of guessing the answer is higher.

The key step is to define the amount of information gained by determining
an event $e$ ($I(e)$) (in our case, finding out what the target is).

$$
I(e) = - log_{2} \,p(e)
$$

Read this as $I(e)$, the information needed to determine $e$, 
is equal to the negative of the log of the probability of $e$ (to the base of 2).
The unit is **bits**, binary choices.
For example, suppose we are talking about the information needed to determine
that a coin flip X has resulted in heads.  Assuming a fair coin, the probability of
a heads outcome is 1/2, so
$$
I(X=\text{heads}) = -log_{2} 1/2 = - (- 1) = 1 \text{ bit}
$$
So 1 bit of information is needed to determine that X=heads.  Or suppose
we are rolling a 6-sided die (Y).  The probability that the outcome
is 3 is 1/6, so $I(Y=\text{3})$ is:

In [1]:
import math

- math.log(1/6,2)

2.584962500721156

Since logs of probabilities are negative, determining events with lower
probability takes more information.   Hence the outcome of the die toss carries more information
than the outcome of the coin flip.

We can now quantify the intuitions
above.  In the scenario above we start in a situation
with 24 equally probable candidates.  That means the information
needed to determine any one of them is

In [12]:
Ic = - math.log(1/24,2)
Ic

4.584962500721157

Then we assumed that the next guess narrowed things down to a group of 3 members.  That means the information
needed to determine any one of them  went down: 

In [13]:
Is = - math.log(1/3,2)
Is

1.5849625007211563

So the information gain achieved by the guess that got us from a set of equally likely candidates
of size 24 to a set of candidates of size 3 is:

In [14]:
#4.584962500721157 - 1.5849625007211563
Ic - Is

3.000000000000001

That's 3 bits, give or take a floating point twitch.  Note that when we went from a set of size 24 to a set
of size 3, we shrunk the number of possibilities by a factor of 8.  The 3 bits comes from the fact
that $\log 8 = 3$.  Or to put it another way, halving the number of possibilities takes one bit
(as it did when we flipped the coin and went from 2 possible outcomes to 1).  Halving the number of possibilties
again takes another bit.  Halving them a third time takes a third bit.  So reducing the number of
possibilities by a factor of 8 takes 3 bits.  

That's the key idea about what 1 bit of information means: if we are in a state in which'there are N equally likely possibilities and we move to a state in which there are N/2 possibilities then we have received
1 bit of information. If we want to end up  in  a state in which there is only 1 possibility
it will take $\log_{2} N$ bits ($\log_{2} N$ successive halvings of the number of possibilities).

What about reducing the number of equally likely possibilities by 1? The amount of information that will
take depends on N, the total number of possibilities.  If N=2, it's 1 bit; if N is greater
it will be less, in general it's

$$
\log_{2} \frac{N}{N-1} = \log_{2} {N} - \log_{2} ({N-1}) 
$$

Let's summarize what we just did and introduce some
notation.  Call the original set with 24 possibilities C.
And let's use $I_{C}$ for the information required to determine any single member of C
assuming they're all equally likely, that is,

$$
\begin{array}{lcl}
\text{I}_{C} &=& - \log_{2} \frac{1}{\mid \text{C} \mid}
             & = & \log_{2} \mid \text{C} \mid \\
&=& \log_{2}(24) = 4.584962500721157
\end{array}
$$

Then we assumed that the next guess narrowed things down to a set of 3 members.
Let's call that set with 3 members $s$
And let's use $I_{s}$ for the information required to determine 1 member of $s$:

$$
\begin{array}{lcl}
\text{I}_{s} &=& - \log_{2} \frac{1}{\mid \text{s} \mid}
             & = & \log_{2} \mid s \mid \\
&=& \log_{2}(3) = 1.5849625007211563
\end{array}
$$

The information gained by that guess is:

$$
\text{I}_{C}  - \text{I}_{s} = 3 \text{ bits}.
$$

More generally, we define the Information Gain of a partition of a set C (the set
of candidates) as the average of the information gains of 
narrowing the possibilities down to $s_{i}$ for each $s_{i}$
in the partition:

$$
\begin{array}{llcl}
(a) &\text{IG}(\pi) & = & \sum_{s \in \pi} p_{\pi}\,(s_{i})\left \lbrack \text{I}_{C} - \text{I}_{s_{i}} \right \rbrack \\
(b)&                & = & \sum_{s \in \pi} p_{\pi}(s_{i})\,\text{I}_{C} - \sum_{s \in \pi}p_{\pi}(s_{i})\cdot \text{I}_{s_{i}}\\
(c)&                & = & \text{I}_{C} - \sum_{s \in \pi}p_{\pi}(s_{i})\cdot \text{I}_{s_{i}}\\
\end{array}
$$


Here,the weighting of each set in the average is given by its probability. The probability
associated with a set  $s_{i}$ in the partition $\pi$ is simply the probability that a randomly
chosen member of C  will be in $s_{i}$.  Writing that out :

$$
p_{\pi}(s_{i}) = \frac{\mid s_{i} \mid}{\mid \text{C} \mid}
$$.

Given our definition of information for a partition,
let's calculate the information gain for the two cases above.

In [15]:
# A:  2 groups of size 3  4 groups of size 2 10 groups of size 1
# the weighted gains 
GainsA = \
2 * -math.log(1/3,2) * 3/24 + \
4 * -math.log(1/2,2) * 2/24 + \
10 * -math.log(1/1,2) * 1/24
GainsA 

0.7295739585136224

So given this partioning  on average it will take .73 yes-no questions to determine target.  How is that possible?  Bear in mind that this is the **average** number of questions, and that for 10 of the partitions, amounting to 
10/24 of the probability mass, 0 questions are required:

In [9]:
-math.log(1/1,2)

-0.0

Our final information gain:

In [18]:
Ic - GainsA

3.8553885422075345

Computing the average partition size for partition A:

In [8]:
# average partitio size
(2*3 + 4*2 + 10 * 1)/16

1.5

Turning to Partition B:

In [20]:
# B:  1 groups of size 3  8 groups of size 2 5 groups of size 1
# the weighted gains 
GainsB = \
1 * -math.log(1/3,2) * 3/24 + \
8 * -math.log(1/2,2) * 2/24 + \
5 * -math.log(1/1,2) * 1/24
GainsB 

0.8647869792568111

In [21]:
Ic - GainsB

3.7201755214643457

In [17]:
# average partition size for Partition B
(1*3 + 8*2 + 5 * 1)/14

1.7142857142857142

So both metrics agree that Partition A should be preferred.

#### Coding exercise

**Part One**

Write a function that scores a partition using the above definition of information
gain for a partition.  Assume a partition is a container of
sets.  Assume the sets are disjoint so that we have a genuine
partition.  The function `score_partition` has the signature

```python
score_partition(partition)
```

and returns a floating point number that is the average information gain for
that partition.

Check your function  by seeing if you get the same answer for partitions
like A and B as we got in  the calculations above.
You will have to cook up your own partitions to do the test.
Bear in mind that the set members don't matter.  What matters
is the total number of members in C (24, for partitions A and B) and the sizes of the sets
C is partitioned into.  Also bear in mind the partition sets
must not overlap.

In [49]:
import math

#  Put the definition of score_partition here



In [48]:
# Test your score_partition function on partition lkike partition A and partition B above
# A:  2 groups of size 3  4 groups of size 2 10 groups of size 1
#part_A = 
    
print("A", score_partition(part_A))

# B:  1 groups of size 3  8 groups of size 2 5 groups of size 1
#part_B = 
    
print("B", score_partition(part_B))

A 3.8553885422075345
B 3.7201755214643457


**Part Two**

Take your partition scorer out for a spin by using it to find the best Wordl opening word.

The situation:  All possible words are your candidates. Consider each as a potential guess
and use it to partition the set of candidates.  Score your partition using your partition scorer,
then rank words by partition score.  Report the highest ranking word.  For fun you should
also report the lowest ranking word to answer this too infrequently asked question:  "What is the worst possible
Wordl opener?" (Keep these words around; they make great target words to test on when you finish building
your WordlBot).

1. To partition the set of candidates, use the function `color_guess` provided to you below for free.
   It handles the case of guesses that have duplicate letters correctly, which is a wrinkle
   that introduces some slightly tricky logic. It also represents colorings as strings of length 5,
   which means they can be keys in a dictionary.  Hint, hint. You will probably want to write a function
   `get_partition` that loops through the candidates and assigns each to a partition set 
   with `color_guess`.  Perfectly oblique to everything, you may want to know about `defaultdict`
   in the `collections` module, which makes it easy to create and maintain a dictionary whose
   values are sets.  Its use is illustrated in the score_partition2 functoon below
2. Use the wordlist provided to you in the next cell as your set of all possible 5-letter English words.
   It's far from perfect, but it's a decent simulation of the unknown set of actual Wordl solutions.

In [96]:
import pandas as pd

def get_wordl_words():
    df = pd.read_csv("wordlebot_words.txt",header=None,names=["Word"])
    words = df["Word"].values
    words = [w for w in words if w.isalpha()]
    print(len(words), "words loaded")
    return words

words = get_wordl_words()

3204 words loaded


In [97]:
from collections import defaultdict

def color_guess (target, guess,wd_len=5):
    """
    This version appears to fix the duplicate
    letters bug.
    """
    def wd_dict(wd):
        dd = defaultdict(set)
        for (i,l) in enumerate(wd):
            dd[l].add(i)
        return dd

    coloring = list('k'*5)
    tgt_dict,guess_dict = wd_dict(target),wd_dict(guess)
    for let in set(guess):
        gs_toks,tgt_toks = guess_dict[let],tgt_dict[let]
        gs = gs_toks & tgt_toks
        hits, max_hits = 0, len(tgt_toks)
        for gs_i in gs:
            coloring[gs_i] = 'g'
            hits += 1
        # We have yellows to assign
        for gs_i in gs_toks - gs:
            if tgt_toks and hits < max_hits:
                coloring[gs_i] = 'y'
                hits += 1
    return ''.join((coloring))


def score_partition2(partition):
    """
    Alternative partition scoring metric.  You can use this a s placeholder for 
    the information theoretic version, while debugging.  It has the same type of inputs
    and outputs.  Of course the nu,bers output are different.
    
    Uses average sz of partition sets as score
    """
    Gains = 0
    for s in partition:
        Gains += len(s)
    return Gains/len(partition)

Solution code 1:

In [98]:
###  Your code here

Worst and best words:

## Part Three

You have almost written a WordlBot.  

Want to finish?  Your Bot can operate in two modes.

1.  Write a while loop that is given a value for an initial guess, then waits to have a coloring input before outputting the next guess. Each coloring that is input should update the set of current candidate words.  Updating the current candidates will be quite straightforward if you still have the partition induced by your last guess, especially if you implemented your partition as a dictionary that maps a coloring to the set of words compatible with that coloring. Where are the colorings coming from?  You're playing Wordl online. Your Wordl skill scores should skyrocket.
2.  Give the Bot a target word (a solution) and let it play on its own (it can now generate its own colorings).  You can now compare your Wordl games with its games.  You can also search for the hardest Wordl words, the ones that make it take 4 or more guesses.  

## Given code (either from earlier in this NB or new for this problem)

In [161]:
import math
import pandas as pd
from collections import defaultdict
import random

def get_wordl_words():
    url= "https://raw.githubusercontent.com/gawron/python-for-social-science/refs/heads/master/" +\
    "text_classification/wordlebot_words.txt"
    #df = pd.read_csv("wordlebot_words.txt",header=None,names=["Word"])
    df = pd.read_csv(url,header=None,names=["Word"])
    words = df["Word"].values
    words = [w for w in words if w.isalpha()]
    print(len(words), "words loaded")
    return words

def get_partition_dict(word, candidates):
    partition_dict = defaultdict(set)
    for target in candidates:
        partition_dict[color_guess (target, word)].add(target)
    return partition_dict


def color_guess (target, guess,wd_len=5):
    """
    This version appears to fix the duplicate
    letters bug.
    """
    def wd_dict(wd):
        dd = defaultdict(set)
        for (i,l) in enumerate(wd):
            dd[l].add(i)
        return dd

    coloring = list('k'*5)
    tgt_dict,guess_dict = wd_dict(target),wd_dict(guess)
    for let in set(guess):
        gs_toks,tgt_toks = guess_dict[let],tgt_dict[let]
        gs = gs_toks & tgt_toks
        hits, max_hits = 0, len(tgt_toks)
        for gs_i in gs:
            coloring[gs_i] = 'g'
            hits += 1
        # We have yellows to assign
        for gs_i in gs_toks - gs:
            if tgt_toks and hits < max_hits:
                coloring[gs_i] = 'y'
                hits += 1
    return ''.join((coloring))


def score_partition(partition):
    """
    Use information gain to score a partition.
    """
    C = {s for S in partition for s in S}
    N = len(C)
    Ic = math.log(N,2)
    Gains = 0
    for S in partition:
        sz = len(S)
        Gains += (math.log(sz,2)*(sz/N))
    return Ic-Gains

def score_partition2(partition):
    """
    Alternative scoring metric.  Use average sz of partition sets as score
    """
    Gains = 0
    for s in partition:
        Gains += len(s)
    return Gains/len(partition)

In [ ]:
## Solution

## Your code here

## Introducing entropy

In tnis small section we discuss the relationship of our definition
of Information Gain to some standard concepts of Information Theory.

We were able to greatly simplify the discussion of Information Gain above because we
were dealing with uniform distributions, quite reasonably so.  From
the point of view of a Wordl player every 5-letter has to have an equal
chance being the target. So we did quite well by defining something
we called $\text{I}_{s}$, the information required to determine
any single member of S, which depended only the size of S. 
>Note:  If you're a follower of the WordleBot analyses, you may have noticed that the Bot doesn't actually respect this assumption when assigning probabilities to solutions.  Some words have very low probabilities of being targets. It's not at all clear what justifies these numbers and I'm not sure how a Wordl player could know them, so I've left this out of the discussion.  As will become clear below, it made life simpler.

But Information Theory is designed  to deal with a general
probability distributions, where all the events may have different
probabilities, so the average information required to determine
an event for a set of events has a slightly more complicated definition.
The term in Information Theory is **Entropy** and the definition of Entropy is:

$$
H(X) = -\sum_{x\in X} p(x) \log_{2} p(x)
$$

Your favorite Information Theory textbook
will call X a random variable (for example, Cover and Thomas 1991 *Elements of Information
Theory*),  but for our purposes
we can think of it as a set of events or values with an associated
probability distribution $p$. The entropy is the probability-weighted average
of the information carried by events in X, also called the **Expected Information**.
Let's apply this definition to our set of candidate words C,
assuming as usual they all have an equal chance of being the target:

$$
\begin{array}{lcl}
H(C) &=& - \sum_{x\in C} p(x) \log_{2} p(x)\\
    & = & \mid C \mid \cdot \frac{1}{\mid C \mid} \log_{2} \mid C \mid   \\
     & = & \log_{2} \mid C \mid  \\
\end{array}
$$

This of course is what we have been calling $I_{C}$.
When all the candidates carry an equal amount of information
the average information is the same as the information
carried by any single candidate. This allowed us to 
compute the entropy of C just by computing the information carried by one
candidate.

The next important concept is conditional entropy.

$$
H(X \mid Y) = \sum_{y\in Y} p(y) \cdot H(X\mid y)
$$

That is the expected conditional entropy of
X given Y, or the average amount of information
carried by an event in X when the value of Y is known.

In our context we  have been talking about
C and various sets in a partition of C,
$\pi$.  We computed the information in 
knowing the Partition Information (PI) as:

$$
\text{PI} = \sum_{s \in \pi}p_{\pi}(s_{i})\cdot \text{I}_{s_{i}}\\
$$

And we now know $\text{I}_{s_{i}}$ is equal to
$H(s_{i})$.  Also, since $s_{i}$ is a subset of C,
for any $x \in C$, the probability of $x$
being the target given that we know it is in $s_{i}$
is the same as the probability of any x in
 $s_{i}$ the target, so we can write $H(s_{i})$ as 
$H(\text{C}\mid s_{i})$.  So we have:

$$
H(C \mid \pi)  = \text{IP} = \sum_{s \in \pi}p_{\pi}(s_{i})\cdot H(\text{C}\mid s_{i})\\
$$

That is, what we've been calling 
the PI is the Conditional Entropy of $C$ given $\pi$,
or the average information needed determine the target when it is known
which of the partitions of $\pi$ the target is in.

When we plug these rewrites back into our definition of
Information Gain, we get the standard definition of
Information Gain

$$
IG(C,\pi) = H(C) - H(C\mid \pi)
$$

This is also equivalent to the **Mutual Information** of C and $\pi$,
written $I(C;\pi)$.


## fragment 1
Spelling this out a bit:  the definition of the amount of information in an event with probability $p$ is

$$
- \log p
$$

There are good reasons for this definition.  For example, teh minus sign
tells us the more improbable an event is teh more information it carries.
Also the infomeation
carried by two independent events is just 

$$
- \log pq = - \log p - \log q.
$$

Thus, the information content of two independent events is just the sum of their respective
information contents.  There are other motivations we leave out for lack of space.


## fragment 2

Entropy can be thought of as measuring how many binary choices on
average we need to determine one event given a probability
distribution over events. The units are called bits. The worst case is when all the events are
equally probable.  The number above means that to discriminate among 24 equally probable candidates
on average it will take 4.585 or so binary choices (answers to yes-no questions).  For example,
supposing it's a number between 1 and 24, and supposing we get a series of yes answers, we
might preoceed as follows

1.  Is it less than 13?
2.  Is it less than 7?
3.  Is it less than 4?

Whereupon we have 3 candidates left to choose among, which will take either 1 or 2 questions,
and on average it will be:

In [6]:
math.log(3,2)

1.5849625007211563

questions.

If you know that the target is in a set
of size 1, you are done.  It takes 0 binary questions to identify the target. That's a
0-entropy distribution.  If you know the target is in a set of size 2, it takes
1 yes-no question to identify the target.  That partition set has an entropy of 1 bit.

$$
\begin{array}{lcl}
1 * 1 * (- \log 1) & = & 0\\
2* .5 * (- \log .5) & = & 1 \\
\end{array}
$$

If you have a set of size 3, as we just saw, on average it takes 1.58 yes-no questions
(= 1.58 bits). 